# 이 쥬피터 노트북은 Keras의 기능을 정확히 알고 이해하기 위해 테스트하기 위해 생성되었습니다.

# Embedding test

In [15]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.constraints import MinMaxNorm
from keras.initializers import RandomUniform

import numpy as np

In [18]:
model = Sequential()
model.add(Embedding(1000, 64, 
#                     embeddings_constraint=MinMaxNorm(min_value=0.0, max_value=1.0, axis=1),
                    embeddings_initializer=RandomUniform(minval=0.0, maxval=1.0),
                    input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
assert output_array.shape == (32, 10, 64)

In [19]:
output_array

array([[[0.09222102, 0.5878209 , 0.18674517, ..., 0.9334879 ,
         0.3783021 , 0.30190313],
        [0.27557755, 0.922205  , 0.59224284, ..., 0.63351727,
         0.31549442, 0.6149924 ],
        [0.1291343 , 0.7746289 , 0.41895533, ..., 0.8637204 ,
         0.94625974, 0.60956407],
        ...,
        [0.13277078, 0.9567723 , 0.28050184, ..., 0.31227982,
         0.45570648, 0.12249959],
        [0.7117882 , 0.43718553, 0.3312415 , ..., 0.04832101,
         0.0699507 , 0.720222  ],
        [0.11619294, 0.251822  , 0.48438454, ..., 0.5212587 ,
         0.9755002 , 0.18777192]],

       [[0.03828454, 0.12515783, 0.24206626, ..., 0.04928219,
         0.04028773, 0.69018126],
        [0.94320023, 0.94497025, 0.74802744, ..., 0.58057475,
         0.5958452 , 0.34266198],
        [0.49543047, 0.31238997, 0.92750704, ..., 0.11330009,
         0.36783922, 0.77677214],
        ...,
        [0.5720235 , 0.24930668, 0.62350464, ..., 0.47211063,
         0.92909706, 0.66098094],
        [0.6

# Lambda Test

In [1]:
from keras.models import Sequential, Model
from keras.layers import Lambda, Input, Concatenate, TimeDistributed, Dense
import keras.backend.tensorflow_backend as K
import numpy as np

Using TensorFlow backend.


In [4]:
seq_length = 3
latent_dim = 2
input_array = np.random.randint(10, size=(1, seq_length, latent_dim))

In [5]:
input_array

array([[[2, 2],
        [6, 3],
        [7, 1]]])

In [6]:
axis = 2
def antirectifier(x):
    
    return x ** 2

def antirectifier_output_shape(x):
    return tuple((x[0], ) + x[1:axis] + (seq_length, ) + x[axis:])

In [7]:
model = Sequential()
model.add(Lambda(antirectifier, antirectifier_output_shape))
# model.add(Lambda(antirectifier))
model.compile(optimizer='rmsprop', loss='mse')


output_array = model.predict(input_array)

In [8]:
for a, b in zip(input_array[0], output_array[0]):
    print(a, '->',  b)

[2 2] -> [4. 4.]
[6 3] -> [36.  9.]
[7 1] -> [49.  1.]


# Repeat Vector Layer

In [9]:
def RepeatVectorLayer(rep, axis):
#     print(axis)
    return lambda x: K.repeat_elements(K.expand_dims(x, axis), rep, axis)
#     return K.repeat_elements(K.expand_dims(x, axis), rep, axis)

In [78]:
inputs = Input(shape=(seq_length, latent_dim))
repeat_d = Lambda(RepeatVectorLayer(seq_length, 2))(inputs)
print(repeat_d.get_shape())
repeat_e = Lambda(RepeatVectorLayer(seq_length, 1))(inputs)
print(repeat_e.get_shape())
outputs = Concatenate()([repeat_d, repeat_e])
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='mse')
output_array = model.predict(input_array)

(?, 3, 3, 2)
(?, 3, 3, 2)


In [11]:
for a, b in zip(input_array[0], output_array[0]):
    print(a, '->',  b)

[2 2] -> [[2. 2. 2. 2.]
 [2. 2. 6. 3.]
 [2. 2. 7. 1.]]
[6 3] -> [[6. 3. 2. 2.]
 [6. 3. 6. 3.]
 [6. 3. 7. 1.]]
[7 1] -> [[7. 1. 2. 2.]
 [7. 1. 6. 3.]
 [7. 1. 7. 1.]]


In [12]:
print(input_array.shape)
print(output_array.shape)

(1, 3, 2)
(1, 3, 3, 4)


# RepeatVector from Keras

In [72]:
from keras.layers import RepeatVector, Reshape, Permute, TimeDistributed, concatenate, merge

In [95]:
inputs = Input(shape=(seq_length, latent_dim))
reshaped_d_inputs = Reshape((inputs.get_shape()[1] * inputs.get_shape()[2], ))(inputs)
print(reshaped_d_inputs.get_shape())
repeat_d_repeat = RepeatVector(seq_length)(reshaped_d_inputs)
repeat_d = Reshape((seq_length, inputs.get_shape()[1], inputs.get_shape()[2]))(repeat_d_repeat)
print(repeat_d.get_shape())


permuted_e = Permute((2, 1))(inputs)
reshaped_e_inputs = Reshape((inputs.get_shape()[1] * inputs.get_shape()[2], ))(permuted_e)
permuted_e_repeat = RepeatVector(seq_length)(reshaped_e_inputs)
repeat_e = Reshape((seq_length, inputs.get_shape()[2], inputs.get_shape()[1]))(permuted_e_repeat)
repeat_e = Permute((1, 3, 2))(repeat_e)
print(repeat_e.get_shape())

outputs = Concatenate(-1)([repeat_d, repeat_e])
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='mse')
output_array = model.predict(input_array)

(?, 6)
(?, 3, 3, 2)
(?, 3, 3, 2)


TypeError: unhashable type: 'Dimension'

In [120]:
inputs = Input(shape=(seq_length, latent_dim))
repeat_d = K.repeat(inputs, 2)
print(repeat_d.get_shape())
# # repeat_d = TimeDistributed(repeat_d)


# reshaped_e_inputs = Reshape((inputs.get_shape()[1] * inputs.get_shape()[2], ))(inputs)
# print(reshaped_e_inputs.get_shape())
# repeat_e =  RepeatVector(seq_length)(reshaped_e_inputs)
# print(repeat_e.get_shape())
# # repeat_e = Reshape((seq_length, seq_length, latent_dim))(repeat_e)

# # outputs = Concatenate()([repeat_d, repeat_e])
# outputs = np.concatenate([repeat_d, repeat_e])

model = Model(inputs=inputs, outputs=repeat_d)
model.compile(optimizer='rmsprop', loss='mse')
output_array = model.predict(input_array)

AssertionError: 

In [82]:
for a, b in zip(input_array[0], output_array[0]):
    print(a, '->',  b)

[2 2] -> [[[2. 2.]
  [6. 3.]
  [7. 1.]]

 [[2. 2.]
  [6. 3.]
  [7. 1.]]

 [[2. 2.]
  [6. 3.]
  [7. 1.]]]


# Attention layer test

In [51]:
inputs = Input(shape=(seq_length, latent_dim))
repeat_d = Lambda(RepeatVectorLayer(seq_length, 2))(inputs)
repeat_e = Lambda(RepeatVectorLayer(seq_length, 1))(inputs)
concat_v = Concatenate()([repeat_d, repeat_e])
dense_score_layer = TimeDistributed(Dense(latent_dim // 2, activation='tanh'))
dense_score = dense_score_layer(concat_v)
outputs = dense_score

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='mse')
print(model.summary())
output_array = model.predict(input_array)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 5, 10)        0                                            
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, 5, 5, 10)     0           input_6[0][0]                    
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, 5, 5, 10)     0           input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_6 (Concatenate)     (None, 5, 5, 20)     0           lambda_11[0][0]                  
                                                                 lambda_12[0][0]                  
__________

In [52]:
layer_outputs = list()
for idx, l in enumerate(model.layers):
    if idx == 0:
        continue
    layer_outputs.append(l.output)
get_3rd_layer_output = K.function([model.layers[0].input],
                                  layer_outputs)
layer_output = get_3rd_layer_output([input_array])

In [53]:
print()
for l_output in layer_output:
    print(l_output[0][0])
    print('')


[[4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]]

[[4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]
 [4. 3. 8. 7. 6. 1. 1. 1. 5. 0.]
 [2. 3. 7. 1. 7. 3. 8. 5. 4. 7.]
 [3. 1. 5. 8. 9. 1. 1. 1. 6. 8.]
 [0. 9. 9. 8. 4. 5. 0. 6. 4. 2.]]

[[4. 9. 2. 5. 5. 9. 6. 0. 6. 0. 4. 9. 2. 5. 5. 9. 6. 0. 6. 0.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0. 4. 3. 8. 7. 6. 1. 1. 1. 5. 0.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0. 2. 3. 7. 1. 7. 3. 8. 5. 4. 7.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0. 3. 1. 5. 8. 9. 1. 1. 1. 6. 8.]
 [4. 9. 2. 5. 5. 9. 6. 0. 6. 0. 0. 9. 9. 8. 4. 5. 0. 6. 4. 2.]]

[[-0.93206555 -1.         -0.9967784   0.9914774   0.9949547 ]
 [-0.9999221  -1.         -0.9999996   0.79444325 -0.5474762 ]
 [-0.9999033  -1.         -0.9999963   0.28762528 -0.9344036 ]
 [-0.99998504 -1.         -1.          0.95935047 -0.7652957 ]
 [-0.99999994 -1.         -0.99999774 -0.5401838  -0.2644022 ]]



In [46]:
layer_output[0][0][0]

array([[2., 4.],
       [2., 4.],
       [2., 4.],
       [2., 4.],
       [2., 4.]], dtype=float32)